In [80]:
#importing libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [47]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv") 

#data preprocessing of training and test data
# train.isnull().sum()

randoms = np.random.randint(train['Age'].mean() - train['Age'].std(),train['Age'].mean() + train['Age'].std() )
train["Age"] = train["Age"].fillna(randoms)
train["Cabin"] = train["Cabin"].fillna("G6")
train["Embarked"] = train["Embarked"].fillna("S")
train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [48]:
# test.isnull().sum()
random = np.random.randint(test['Age'].mean() - test['Age'].std(),test['Age'].mean() + test['Age'].std() )
test["Age"] = test["Age"].fillna(random)
test["Fare"] = test["Fare"].fillna(np.random.randint(test['Fare'].mean() - test['Fare'].std(),test['Fare'].mean() + test['Fare'].std()))
test["Cabin"] = test["Cabin"].fillna("B57 B59 B63 B66")
test.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [49]:
#deleting irrelevant columns 
train.drop(['Name'], axis='columns')

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,G6,S
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,G6,S
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,male,35.0,0,0,373450,8.0500,G6,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,211536,13.0000,G6,S
887,888,1,1,female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,female,21.0,1,2,W./C. 6607,23.4500,G6,S
889,890,1,1,male,26.0,0,0,111369,30.0000,C148,C


In [50]:
#converting categorical to numerical
# dummy_tic=pd.get_dummies(train["Ticket"])
train['Sex'].replace(['female', 'male'],
                        [1, 2], inplace=True)
test['Sex'].replace(['female', 'male'],
                        [1, 2], inplace=True)
train['Embarked'].replace(['C', 'S','Q'],
                        [1, 2,3], inplace=True)
test['Embarked'].replace(['C', 'S','Q'],
                        [1, 2,3], inplace=True)
train['Cabin'] = LabelEncoder().fit_transform(train['Cabin'])
train['Ticket'] = LabelEncoder().fit_transform(train['Ticket'])
test['Cabin'] = LabelEncoder().fit_transform(test['Cabin'])
test['Ticket'] = LabelEncoder().fit_transform(test['Ticket'])

# pd.concat([train, dummy_tic], axis='columns')
# train.drop(['Ticket'], axis='columns')

In [51]:
#scaling data

x=train[["Pclass","Sex","Age","SibSp","Parch","Fare","Ticket","Cabin","Embarked"]]
y = train['Survived']
# print(len(y))
scaler = StandardScaler()
x = scaler.fit_transform(x)
tests=test.drop(['Name','PassengerId'], axis=1)
# test
test= scaler.transform(tests)


C:\Users\Farida Eleshin\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [53]:
# Implementing Logistic Regression using sklearn
modelLogistic = LogisticRegression(random_state=0)
modelLogistic.fit(x,y)
y_pred= modelLogistic.predict(test)
for i in range(418,len(y)):
    y= y.drop(i)
print(len(y))
print(len(y_pred))
ConfusionMatrix = confusion_matrix(y, y_pred)
print(ConfusionMatrix)

418
418
[[128 127]
 [ 92  71]]


In [55]:
x2=train[["Pclass","Sex","Age","SibSp"]]
y2 = train['Survived']
x2 = scaler.fit_transform(x2)
# test
tests=tests.drop(["Parch","Fare","Ticket","Cabin","Embarked"], axis=1)
tests= scaler.transform(tests)

modelLogistic1 = LogisticRegression(random_state=0)
modelLogistic1.fit(x2,y2)
y2_pred= modelLogistic1.predict(tests)

for i in range(418,len(y2)):
    y2= y2.drop(i)
ConfusionMatrix = confusion_matrix(y2, y2_pred)
print(ConfusionMatrix)

[[150 105]
 [108  55]]


In [14]:
# testss = pd.read_csv("test.csv")
# data = {'PassengerId':testss['PassengerId'],'Survived':y2_pred}
# data = pd.DataFrame(data)
# # data
# data.to_csv('titanic_submission.csv')

array([[ 0.82737724,  0.73769513,  0.51601502, ..., -1.64741882,
        -3.23418652,  2.14282826],
       [ 0.82737724, -1.35557354,  1.42131095, ..., -1.65154958,
        -3.23418652,  0.19857166],
       [-0.36936484,  0.73769513,  2.50766607, ..., -1.63816148,
        -3.23418652,  2.14282826],
       ...,
       [ 0.82737724,  0.73769513,  0.80570972, ..., -1.65030418,
        -3.23418652,  0.19857166],
       [ 0.82737724,  0.73769513,  0.62465053, ..., -1.64631888,
        -3.23418652,  0.19857166],
       [ 0.82737724,  0.73769513,  0.62465053, ..., -1.57504037,
        -3.23418652, -1.74568493]])

# Random Forest Classifier Approach


In [67]:
y = train['Survived']
rand_forest = RandomForestClassifier(random_state = 0)
nums = [*range(30,100,5)]
param_grid = { 
    'n_estimators': nums,
}
cv_rfc = GridSearchCV(estimator=rand_forest, param_grid=param_grid, cv= 5)
cv_rfc.fit(x, y)  
cv_rfc.best_params_

{'n_estimators': 80}

In [68]:
rfc1=RandomForestClassifier(random_state=0, n_estimators= 80)
rfc1.fit(x,y)

RandomForestClassifier(n_estimators=80, random_state=0)

In [69]:
y_predx = rfc1.predict_proba(x)[::,1]
aucx = roc_auc_score(y, y_predx)
aucx

0.9999680439714953

In [79]:
y_pred2 = rfc1.predict_proba(test)[::,1]
# int(y_pred2)
# auc2 = roc_auc_score(y, y_pred2)
# auc2


In [77]:
y_int = []
for num in y_pred2:
    num = int(num)
    y_int.append(num)
#     print(num)

In [78]:
testss = pd.read_csv("test.csv")
data = {'PassengerId':testss['PassengerId'],'Survived':y_int}
data = pd.DataFrame(data)
# data
data.to_csv('titanic_subs.csv')

In [81]:
rand_forestreg = RandomForestRegressor(random_state = 0)
numsreg = [*range(30,100,5)]
param_gridreg = { 
    'n_estimators': numsreg,
}
cv_rfcreg = GridSearchCV(estimator=rand_forestreg, param_grid=param_gridreg, cv= 5)
cv_rfcreg.fit(x, y)  
cv_rfcreg.best_params_

{'n_estimators': 95}

In [82]:
rfc1reg=RandomForestRegressor(random_state=0, n_estimators= 95)
rfc1reg.fit(x,y)

RandomForestRegressor(n_estimators=95, random_state=0)

In [85]:
y_predxreg = rfc1reg.predict(x)
aucxreg = roc_auc_score(y, y_predxreg)
aucxreg

0.999973369976246

In [86]:
y_pred2reg = rfc1.predict(test)

In [87]:
testss = pd.read_csv("test.csv")
data = {'PassengerId':testss['PassengerId'],'Survived':y_pred2reg}
data = pd.DataFrame(data)
# data
data.to_csv('titanic_subsreg.csv')